## State Stabilization

Here the result is as follows: The algorithm optimizes the params, such that the POVM always outputs 1, implying that the measurement leaves the target state invariant. this is what we are indeed seeing when printing the measurement outcome and its probability, when batching however, the optimizer struggles to converge.

Also, lookup here is much better than nn for the same hyperparameters.

This is actually a perfect example, of feedback grape modifying the params so that a certain measurement sequence will always be output because this measurement sequence is the one that is going to lead to the best fidelity

In [1]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

In [2]:
N_cav = 30

In [3]:
def qubit_unitary(alpha):
    """
    TODO: see if alpha, can be sth elser other than scalar, and if the algo understands this
    see if there can be multiple params like alpha and beta input
    """
    return expm(
        -1j
        * (
            alpha * tensor(identity(N_cav), sigmap())
            + alpha.conjugate() * tensor(identity(N_cav), sigmam())
        )
        / 2
    )

In [4]:
def qubit_cavity_unitary(beta):
    return expm(
        -1j
        * (
            beta
            * (
                tensor(destroy(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmap())
            )
            + beta.conjugate()
            * (
                tensor(create(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmam())
            )
        )
        / 2
    )

In [5]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, gamma, delta):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

In [6]:
from feedback_grape.utils.states import coherent

alpha = 3
psi_target = tensor(
    coherent(N_cav, alpha)
    + coherent(N_cav, -alpha)
    + coherent(N_cav, 1j * alpha)
    + coherent(N_cav, -1j * alpha),
    basis(2),
)  # 4-legged state

# Normalize psi_target before constructing rho_target
psi_target = psi_target / jnp.linalg.norm(psi_target)
rho_target = psi_target @ psi_target.conj().T

In [7]:
rho_target.shape

(60, 60)

### It is important to test what the POVM probability is, to check if your state is normalized. if the probability is bounded between 0 and 1 then normalized

In [8]:
# TODO/Question: should one normalize within the optimization just in case?
from feedback_grape.utils.povm import (
    _probability_of_a_measurement_outcome_given_a_certain_state,
)

_probability_of_a_measurement_outcome_given_a_certain_state(
    rho_target, 1, povm_measure_operator, [0.058, jnp.pi / 2]
)

Array(0.70390199, dtype=float64)

In [9]:
from feedback_grape.utils.fidelity import fidelity

print(fidelity(U_final=rho_target, C_target=rho_target, type="density"))

1.0000000079954297


### Without dissipation

In [10]:
result = optimize_pulse_with_feedback(
    U_0=rho_target,
    C_target=rho_target,
    parameterized_gates=[
        povm_measure_operator,
        qubit_unitary,
        qubit_cavity_unitary,
    ],
    measurement_indices=[0],
    initial_params={
        "POVM": [0.058, jnp.pi / 2],
        "U_q": [jnp.pi / 3],
        "U_qc": [jnp.pi / 3],
    },
    num_time_steps=1,
    mode="lookup",
    goal="fidelity",
    optimizer="adam",
    max_iter=1000,
    convergence_threshold=1e-6,
    learning_rate=0.01,
    type="density",
    batch_size=1,
)

Measurement outcome: -1, Probability: 0.2960980115468743, prob_plus: 0.7039019884531257
Measurement outcome: -1, Probability: 0.2960980115468743, prob_plus: 0.7039019884531257
Iteration 0, Loss: 0.148200
Measurement outcome: 1, Probability: 0.6059721804306375, prob_plus: 0.6059721804306375
Measurement outcome: 1, Probability: 0.6059721804306375, prob_plus: 0.6059721804306375
Measurement outcome: -1, Probability: 0.43403587004110866, prob_plus: 0.5659641299588913
Measurement outcome: -1, Probability: 0.43403587004110866, prob_plus: 0.5659641299588913
Measurement outcome: 1, Probability: 0.5119916251203864, prob_plus: 0.5119916251203864
Measurement outcome: 1, Probability: 0.5119916251203864, prob_plus: 0.5119916251203864
Measurement outcome: 1, Probability: 0.48007918043161973, prob_plus: 0.48007918043161973
Measurement outcome: 1, Probability: 0.48007918043161973, prob_plus: 0.48007918043161973
Measurement outcome: -1, Probability: 0.5352050972723035, prob_plus: 0.4647949027276965
Meas

In [11]:
print(result.final_fidelity)

0.9999864698885903


In [12]:
result.optimized_trainable_parameters["initial_params"]

[Array([-5.14923771e-07,  1.46607773e+00], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]

In [13]:
result.returned_params

[[Array([[-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00],
         [-5.14923771e-07,  1.46607773e+00]], dtype=float64),
  Array([[0.01040707],
         [0.01040707],
         [0.01040707],
         [0.01040707],
         [0.01040707],
         [0.01040707],
         [0.01040707],
         [0.01040707],
         [0.01040707],
         [0.01040707]], dtype=float64),
  Array([[1.42996636e-06],
         [1.42996636e-06],
         [1.42996636e-06],
         [1.42996636e-06],
         [1.42996636e-06],
         [1.42996636e-06],
         [1.42996636e-06],
         [1.42996636e-06],
         [1.42996636e-06],
         [1.42996636e-06]], dtype=float64)]

In [14]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, type="density"),
    )

initial fidelity: 1.0000000079954297
fidelity of state 0: 0.9999864698885902
fidelity of state 1: 0.9999864698885902
fidelity of state 2: 0.9999864698885902
fidelity of state 3: 0.9999864698885902
fidelity of state 4: 0.9999864698885902
fidelity of state 5: 0.9999864698885902
fidelity of state 6: 0.9999864698885902
fidelity of state 7: 0.9999864698885902
fidelity of state 8: 0.9999864698885902
fidelity of state 9: 0.9999864698885902


### With Dissipation

In [15]:
# Note if tsave = jnp.linspace(0, 1, 1) = [0.0] then the decay is not applied ?
# because the first time step has the original non decayed state
result = optimize_pulse_with_feedback(
    U_0=rho_target,
    C_target=rho_target,
    parameterized_gates=[
        povm_measure_operator,
        qubit_unitary,
        qubit_cavity_unitary,
    ],
    measurement_indices=[0],
    decay={
        "decay_indices": [0],  # indices of gates before which decay occurs
        # c_ops need to be tensored with the identity operator for the cavity
        # because it is used directly in the lindblad equation
        "c_ops": [tensor(identity(N_cav), jnp.sqrt(0.15) * sigmam())],# c_ops for each decay index
        "tsave": jnp.linspace(0, 1, 2),  # time grid for decay
        "Hamiltonian": None,
    },
    initial_params={
        # if POVM params is 0 and jnp.pi/3 the probability that the measurement outcome is 1 is 1.0 
        # therefore the algorithm trains really well on that measurement outcome
        # this POVM, initial_params, leads to probabilities similar to the fig 6b example
        "POVM": [0.058, jnp.pi / 2],
        "U_q": [jnp.pi / 3],
        "U_qc": [jnp.pi / 3],
    },
    num_time_steps=1,
    mode="lookup",
    goal="fidelity",
    optimizer="adam",
    max_iter=1000,
    convergence_threshold=1e-6,
    learning_rate=0.01,
    type="density",
    batch_size=1,
)

Measurement outcome: -1, Probability: 0.2960980115468741, prob_plus: 0.7039019884531259
Measurement outcome: -1, Probability: 0.2960980115468741, prob_plus: 0.7039019884531259
Iteration 0, Loss: 0.137986
Measurement outcome: 1, Probability: 0.6059721804385564, prob_plus: 0.6059721804385564
Measurement outcome: 1, Probability: 0.6059721804385564, prob_plus: 0.6059721804385564
Measurement outcome: -1, Probability: 0.4322930592580233, prob_plus: 0.5677069407419767
Measurement outcome: -1, Probability: 0.4322930592580233, prob_plus: 0.5677069407419767
Measurement outcome: 1, Probability: 0.5144083578407945, prob_plus: 0.5144083578407945
Measurement outcome: 1, Probability: 0.5144083578407945, prob_plus: 0.5144083578407945
Measurement outcome: 1, Probability: 0.48419976146377963, prob_plus: 0.48419976146377963
Measurement outcome: 1, Probability: 0.48419976146377963, prob_plus: 0.48419976146377963
Measurement outcome: -1, Probability: 0.5287879533475157, prob_plus: 0.4712120466524843
Measur

In [16]:
# 0.7657324286535787
print(result.final_fidelity)

0.9277254984543545


In [17]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, type="density"),
    )

initial fidelity: 1.0000000079954297
fidelity of state 0: 0.9277254984543545
fidelity of state 1: 0.9277254984543545
fidelity of state 2: 0.9277254984543545
fidelity of state 3: 0.9277254984543545
fidelity of state 4: 0.9277254984543545
fidelity of state 5: 0.9277254984543545
fidelity of state 6: 0.9277254984543545
fidelity of state 7: 0.9277254984543545
fidelity of state 8: 0.9277254984543545
fidelity of state 9: 0.9277254984543545


In [18]:
result.returned_params

[[Array([[3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00],
         [3.49424372e-07, 1.46607636e+00]], dtype=float64),
  Array([[0.01360902],
         [0.01360902],
         [0.01360902],
         [0.01360902],
         [0.01360902],
         [0.01360902],
         [0.01360902],
         [0.01360902],
         [0.01360902],
         [0.01360902]], dtype=float64),
  Array([[-1.86010706e-08],
         [-1.86010706e-08],
         [-1.86010706e-08],
         [-1.86010706e-08],
         [-1.86010706e-08],
         [-1.86010706e-08],
         [-1.86010706e-08],
         [-1.86010706e-08],
         [-1.86010706e-08],
         [-1.86010706e-08]], dtype=float64)]]

In [19]:
result.optimized_trainable_parameters["initial_params"]

[Array([3.49424372e-07, 1.46607636e+00], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]

In [20]:
c_ops = (
    [  # c_ops for each decay index
        [
            tensor(identity(N_cav), jnp.sqrt(0.01) * sigmam()),
        ],
        ["hi"],
    ],
)